## **Script do Pipeline de Carga das Cotações das Ações negociadas na Bolsa (Ibovespa/B3)**

Este é um pipeline Python para carregar dados de cotações de ações negociadas na Bolsa de Valores (B3) do Brasil. \
O Pipeline consolida as informações e as exporta para um arquivo Excel. \
Os detalhes das ações que você negocia na bolsa podem ser acessadas pela **Nota de Corretagem**, emitida pela corretora.\
A nota é um extrato das operações realizadas na Bolsa de Valores e contém detalhes como: 
* Espécie da operação
* Quantidade de títulos
* Preço
* Data do pregão
* Valor da negociação
* Corretagem cobrada
* Emolumentos devidos
* Mercado em que a operação foi realizada
* Se a operação foi de compra (C) ou venda (V)
* Tipo de mercado em que a operação foi realizada
* Especificações do título 


#### **Funcionalidades do Pipeline:** 
* Carregar detalhes das ações negociadas a partir de um arquivo Excel.
* Obter informações de setor e segmento das ações utilizando a biblioteca tradingcomdados.
* Carregar o histórico de cotações e dividendos das ações utilizando a biblioteca yahooquery.
* Consolidar os dados das ações, histórico de cotações e dividendos em um único DataFrame.
* Exportar os dados consolidados para um arquivo Excel.


#### **Funções criadas para execução do Pipeline:**
* **run_pipeline:**             Função principal que coordena todo o processo.
* **load_acoes_ibov:**          Carrega os dados das ações do arquivo Excel (Input).
* **get_sector_segment:**       Obtém informações de setor e segmento das ações.
* **load_historical_data:**     Carrega o histórico de cotações e dividendos das ações.
* **consolidate_data:**         Consolida os dados das ações.
* **export_to_excel:**          Exporta os dados consolidados para um arquivo Excel (Output).

#### **Bibliotecas utilizadas:**
* **pandas** -> biblioteca utilizada para armazenar os dados do Pipeline
* **tradingcomdados** -> obtem detalhes de setores econômicos, subsetores e segmentos de mercado, para ativos financeiros negociados na bolsa
* **yahooquery** -> para obter dados históricos de cotações e dividendos das ações.

#### **Diretórios e arquivos dependentes e gerados (Input / Output):**
* **Inputs/MinhasAcoes.xlsx:** Arquivo de entrada contendo os detalhes das ações negociadas.
* **Outputs/MinhasCotacoes.xlsx:** Arquivo de saída onde os dados consolidados são exportados.

#### **O Pipeline Carregará o arquivo com layout (INPUT):**
* **TITULO** CORRETORA = Descrição do Titulo na corretora
* **CODIGO** = Código da ação para negociação na Bolsa
* **COTAS** = Quantidade de fração/Cotas compradas
* **VALOR** = Valor pago na Cota/Fração da Ação
* **DATA COMPRA** = Data da compra da Cota/Fração 

#### **O Pipeline extrairá o arquivo com layout (OUTPUT):**
* **SETORECONOMICO**    = Descrição do Setor ecônomico da ação
* **SUBSETOR**          = Descrição do subsetor ecônomico da ação
* **SEGMENTO**          = Descrição do segmento ecônomico da ação
* **TITULO**            = Código do título da acão
* **ACAO**              = Código da ação negociada na Bolsa de Valores (B3)
* **QTDECOTAS**         = Quantidade de Cotas/Fração da ação
* **VALOR**             = Valor de compra pago na ação
* **DATACOMPRA**        = Data da compra da Cota/Fração
* **DATACOTACAO**       = Data da cotação da Cota/Fração
* **VALORABERTURA**     = Valor de abertura que a Cota/Fração iniciou no dia da cotação
* **MAIORCOTACAO**      = Maior valor/pico que a Cota/Fração da ação chegou no dia da cotação
* **MENORCOTACAO**      = Menor valor que a Cota/Fração da ação chegou no dia da cotação
* **VALORAJUSTADO**     = Valor final da Cota/Fração da ação fechou no dia da cotação
* **DATAPROCESSAMENTO** = Data de processamento/excecução
* **VALORDIVIDENDO**    = Valor do dividendo 
* **FLAG_DIVIDENDOS**   = Data de pagto do dividendo
* **ANOCOTACAO**   = Ano do pagto do dividendo
* **DIVIDENDOSANO**   = Flag do pagto do dividendo no Ano

#### **Observações:** 

O Pipeline não acessa nenhuma plataforma para negociação das suas ações. Ele apenas busca as cotações divugadas no mercado pela B3. \
Em nenhum momento será solicitado nenhum dado pessoal ou login e senha de seus Brokers.


In [1]:
try: 
  import warnings    
  warnings.filterwarnings("ignore")
except ImportError:
   %pip install warnings
   import warnings   
   warnings.filterwarnings("ignore")

try:
  import pandas as pd
except ImportError:
    %pip install pandas
    import pandas

try:
  from tradingcomdados import alternative_data as ad
except ImportError:
    %pip install tradingcomdados
    from tradingcomdados import alternative_data as ad

try:
   from yahooquery import Ticker
except ImportError:
   %pip install yahooquery --upgrade --no-cache-dir
   from yahooquery import Ticker

try:
  import datetime as dt
except ImportError:
   %pip install datetime
   import datetime as dt

**Carrega os detalhes das ações negociadas a partir de um arquivo Excel.**
Args:
- file_path (str): Caminho do arquivo Excel contendo os detalhes das ações negociadas.

Returns:
- Tuple[pd.DataFrame, pd.DataFrame]: DataFrames contendo as ações carregadas e as não localizadas.

**Layout do arquivo INPUT:** 
* **TITULO** CORRETORA = Descrição do Titulo na corretora
* **CODIGO** = Código da ação para negociação na Bolsa
* **COTAS** = Quantidade de fração/Cotas compradas
* **VALOR** = Valor pago na Cota/Fração da Ação
* **DATA COMPRA** = Data da compra da Cota/Fração 

In [2]:
def load_acoes_ibov(file):
    """Carrega os dados das ações do arquivo Excel."""
    return pd.read_excel(file, sheet_name='Planilha1')

**Obtém informações de setor e segmento das ações.**

Args:
- acoes_df (pd.DataFrame): DataFrame contendo as ações negociadas.

Returns:
- pd.DataFrame: DataFrame com informações de setor e segmento adicionadas às ações.

In [3]:
def get_sector_segment(acoes_ibov):
    """Obtém informações de setor e segmento das ações."""
    dfAcoes = pd.DataFrame(columns=['SETORECONOMICO', 'SUBSETOR', 'SEGMENTO', 'EMPRESA', 'TITULO', 'ACAO', 'QTDECOTAS', 'VALOR', 'DATACOMPRA'])
    dfAcoesNaoCarregadas = pd.DataFrame(columns=['TITULO', 'CODIGO', 'COTAS', 'VALOR', 'DATA COMPRA'])

    for idx, row in acoes_ibov.iterrows():
        titulo, acao, cotas, valor, dtcompra = row['TITULO CORRETORA'], row['CODIGO'], row['COTAS'], row['VALOR'], row['DATA']
        try:
            dfSect = ad.get_sectors(acao)
            setor, subsetor, segmento, empresa = dfSect.iloc[0][['SETOR ECONÔMICO', 'SUBSETOR', 'SEGMENTO', 'NOME NO PREGÃO']]
            dfAcoes.loc[idx] = [setor, subsetor, segmento, empresa, titulo, acao, cotas, valor, dtcompra]
        except:
            dfAcoesNaoCarregadas.loc[idx] = [titulo, acao, cotas, valor, dtcompra]

    return dfAcoes, dfAcoesNaoCarregadas

**Carrega o histórico de cotações e dividendos das ações**

Args:
- acoes_df (pd.DataFrame): DataFrame contendo as ações negociadas.

Returns:
- Tuple[pd.DataFrame, pd.DataFrame]: DataFrames contendo o histórico de cotações e de dividendos.

In [4]:
def load_historical_data(dfAcoes):
    """Carrega o histórico de cotações e dividendos das ações."""
    historico_cotacao = pd.DataFrame(columns=['ACAO', 'DATACOTACAO', 'VALORABERTURA', 'MAIORCOTACAO', 'MENORCOTACAO', 'VALORFECHAMENTO', 'VALORAJUSTADO', 'DATAPROCESSAMENTO'])
    dividendos = pd.DataFrame(columns=['ACAO', 'DATACOTACAO', 'VALORDIVIDENDO', 'FLAG_DIVIDENDOS'])

    end_date = dt.datetime.today().strftime('%Y-%m-%d')

    for _, row in dfAcoes.iterrows():
        tkt = row['ACAO'] + '.SA'
        dtcompra = pd.to_datetime(row['DATACOMPRA'], format='%Y-%m-%d')

        hist = Ticker(tkt).history(start=dtcompra, end=end_date).reset_index()
        if not hist.empty:
            hist['ACAO'] = tkt.replace('.SA', '')
            hist['DATACOTACAO'] = pd.to_datetime(hist['date'], format='%Y-%m-%d')
            hist['DATAPROCESSAMENTO'] = pd.to_datetime(end_date)
            hist = hist.rename(columns={'open': 'VALORABERTURA', 'high': 'MAIORCOTACAO', 'low': 'MENORCOTACAO', 
                                        'close': 'VALORFECHAMENTO', 'adjclose': 'VALORAJUSTADO'})
            hist['VALORABERTURA']       = round(hist['VALORABERTURA'],2)
            hist['MAIORCOTACAO']        = round(hist['MAIORCOTACAO'],2)      
            hist['MENORCOTACAO']        = round(hist['MENORCOTACAO'],2)      
            hist['VALORFECHAMENTO']     = round(hist['VALORFECHAMENTO'],2)      
            hist['VALORAJUSTADO']       = round(hist['VALORAJUSTADO'],2)            
            historico_cotacao = pd.concat([historico_cotacao, hist[['ACAO', 'DATACOTACAO', 'VALORABERTURA', 'MAIORCOTACAO', 'MENORCOTACAO', 'VALORFECHAMENTO', 'VALORAJUSTADO', 'DATAPROCESSAMENTO']]])

        hst_dvd = Ticker(tkt).dividend_history(start=dtcompra, end=end_date).reset_index()
        if not hst_dvd.empty:
            hst_dvd['ACAO'] = tkt.replace('.SA', '')
            hst_dvd['DATACOTACAO'] = pd.to_datetime(hst_dvd['date'], format='%Y-%m-%d')
            hst_dvd['FLAG_DIVIDENDOS'] = 'Sim'
            hst_dvd['dividends'] = round(hst_dvd['dividends'],2)
            dividendos = pd.concat([dividendos, hst_dvd[['ACAO', 'DATACOTACAO', 'dividends', 'FLAG_DIVIDENDOS']].rename(columns={'dividends': 'VALORDIVIDENDO'})])

    return historico_cotacao, dividendos

**Consolida os dados das ações, histórico de cotações e dividendos.**

Args:
- acoes_df (pd.DataFrame): DataFrame contendo as ações negociadas.
- cotacoes_df (pd.DataFrame): DataFrame contendo o histórico de cotações das ações.
- dividendos_df (pd.DataFrame): DataFrame contendo o histórico de dividendos das ações.

Returns:
- pd.DataFrame: DataFrame contendo os dados consolidados das ações.

In [5]:
def consolidate_data(dfAcoes, historico_cotacao, dividendos):
    """Consolida os dados das ações."""
    CotacaoConsolidadas = pd.merge(dfAcoes, historico_cotacao, on='ACAO')
    CotacaoConsolidadas = pd.merge(CotacaoConsolidadas, dividendos, on=['ACAO','DATACOTACAO'], how='left')

    # Tratamento de valores nulos
    CotacaoConsolidadas['VALORDIVIDENDO'] = CotacaoConsolidadas['VALORDIVIDENDO'].fillna(0)
    CotacaoConsolidadas['FLAG_DIVIDENDOS'] = CotacaoConsolidadas['FLAG_DIVIDENDOS'].fillna('Não')
    CotacaoConsolidadas['ANOCOTACAO'] = pd.DatetimeIndex(CotacaoConsolidadas['DATACOTACAO']).year

    divAno = CotacaoConsolidadas[CotacaoConsolidadas['FLAG_DIVIDENDOS'] == 'Sim'][['ACAO', 'DATACOTACAO', 'FLAG_DIVIDENDOS']]
    divAno['ANOCOTACAO'] = pd.DatetimeIndex(divAno['DATACOTACAO']).year
    divAno = divAno.drop("DATACOTACAO", axis='columns').rename(columns={'FLAG_DIVIDENDOS': 'DIVIDENDOSANO'}).drop_duplicates().sort_values(by=['ACAO'])

    CotacaoConsolidadas = pd.merge(CotacaoConsolidadas, divAno, on=['ACAO', 'ANOCOTACAO'], how='left')
    CotacaoConsolidadas['DIVIDENDOSANO'] = CotacaoConsolidadas['DIVIDENDOSANO'].fillna('Não')

    return CotacaoConsolidadas

**Exporta os dados consolidados para um arquivo Excel.**

Args:
- data_df (pd.DataFrame): DataFrame contendo os dados consolidados das ações.
- not_found_df (pd.DataFrame): DataFrame contendo as ações não localizadas.
- output_file (str): Caminho do arquivo Excel de saída para armazenar os dados consolidados.

Returns:
- None

In [6]:
def export_to_excel(CotacaoConsolidadas, dfAcoesNaoCarregadas, outputname):
    """Exporta os dados consolidados para um arquivo Excel."""
    with pd.ExcelWriter(outputname, engine='xlsxwriter') as writer:
        CotacaoConsolidadas.to_excel(writer, sheet_name='Cotação das Ações', index=False)
        dfAcoesNaoCarregadas.to_excel(writer, sheet_name='Titulos não Localizados', index=False)

**A função principal que coordena a execução do pipeline, chamando outras funções em sequência**

Args:
- input_file (str): Caminho do arquivo Excel contendo os detalhes das ações negociadas.
- output_file (str): Caminho do arquivo Excel de saída para armazenar os dados consolidados.

Returns:
- None

In [7]:
def run_pipeline(file, outputname):
    """Executa o pipeline completo."""
    acoes_ibov = load_acoes_ibov(file)
    dfAcoes, dfAcoesNaoCarregadas = get_sector_segment(acoes_ibov)
    historico_cotacao, dividendos = load_historical_data(dfAcoes)
    CotacaoConsolidadas = consolidate_data(dfAcoes, historico_cotacao, dividendos)
    CotacaoConsolidadas = CotacaoConsolidadas.rename(columns={'VALOR': 'VALORCOMPRA'})
    export_to_excel(CotacaoConsolidadas, dfAcoesNaoCarregadas, outputname)

**Execução do Pipeline para carga das cotações das ações e dividendos**

**Layout:** 
* **SETORECONOMICO**    = Descrição do Setor ecônomico da ação
* **SUBSETOR**          = Descrição do subsetor ecônomico da ação
* **SEGMENTO**          = Descrição do segmento ecônomico da ação
* **TITULO**            = Código do título da acão
* **ACAO**              = Código da ação negociada na Bolsa de Valores (B3)
* **QTDECOTAS**         = Quantidade de Cotas/Fração da ação
* **VALOR**             = Valor de compra pago na ação
* **DATACOMPRA**        = Data da compra da Cota/Fração
* **DATACOTACAO**       = Data da cotação da Cota/Fração
* **VALORABERTURA**     = Valor de abertura que a Cota/Fração iniciou no dia da cotação
* **MAIORCOTACAO**      = Maior valor/pico que a Cota/Fração da ação chegou no dia da cotação
* **MENORCOTACAO**      = Menor valor que a Cota/Fração da ação chegou no dia da cotação
* **VALORAJUSTADO**     = Valor final da Cota/Fração da ação fechou no dia da cotação
* **DATAPROCESSAMENTO** = Data de processamento/excecução
* **VALORDIVIDENDO**    = Valor do dividendo 
* **FLAG_DIVIDENDOS**   = Data de pagto do dividendo
* **ANOCOTACAO**   = Ano do pagto do dividendo
* **DIVIDENDOSANO**   = Flag do pagto do dividendo no Ano

In [8]:
# Executar o pipeline
file = 'Inputs/MinhasAcoes.xlsx'
outputname = 'Outputs/MinhasCotacoes.xlsx'
run_pipeline(file, outputname)